<a href="https://colab.research.google.com/github/gimb99/PDH_INTEGRADOR_GRUPAL_X/blob/develop/ti_ctg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📥 Paso 2.1 – Ingesta de documentos PDF y traducción automática

Este paso carga los documentos técnicos desde el disco local.
Se leen PDFs tanto en español como en inglés.  
Los documentos en inglés se traducen automáticamente al español con un modelo de Hugging Face.
El resultado final es un corpus unificado en español que se utilizará en los siguientes pasos del sistema RAG.


In [ ]:
# 📌 Paso 2.1 – Ingesta y traducción automática de documentos PDF

# ========================
# ✅ Librerías necesarias
# ========================
!pip install --upgrade langchain langchain-community langchain-core langchain-text-splitters transformers pypdf pymupdf  # Upgrade core libraries
!pip install -U huggingface_hub# Install compatible huggingface_hub
from langchain_community.document_loaders import PyMuPDFLoader  # Para cargar PDFs

from transformers import pipeline                                # Para traducir texto
import os                                                        # Para acceder a archivos en disco


!pip install chromadb
from langchain_community.vectorstores import Chroma

ERROR: Invalid requirement: 'huggingface_hub#': Expected end or semicolon (after name and no valid version specifier)
    huggingface_hub#
                   ^


In [ ]:
# ========================
# ✅ Modelo de traducción
# ========================
# Este modelo traduce texto de inglés a español usando Hugging Face
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [ ]:
# Paso 2.1 – INGESTA DE DOCUMENTOS: Carga de archivos PDF desde Google Drive

# ======================================
# 🔧 1. Montamos Google Drive (solo en Colab)
# ======================================
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ======================================
# 📁 2. Definimos las carpetas del corpus
# ======================================
carpetas_es = "/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base"

carpeta_en = "/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/complementos_tecnicos"


In [ ]:
# ======================================
# 📚 3. Función para cargar documentos PDF
# ======================================
from langchain_community.document_loaders import PyMuPDFLoader
import os

def cargar_documentos_desde_carpeta(rutas_carpeta):
    documentos = []
    for ruta in rutas_carpeta:
        for archivo in os.listdir(ruta):
            if archivo.endswith(".pdf"):
                ruta_completa = os.path.join(ruta, archivo)
                loader = PyMuPDFLoader(ruta_completa)
                documentos_pdf = loader.load()
                documentos.extend(documentos_pdf)
    return documentos

In [ ]:
# ======================================
# 🌍 4. Función para traducir texto de inglés a español
# ======================================
from transformers import MarianMTModel, MarianTokenizer
import torch

# Modelo para traducir de inglés a español
modelo_trad = "Helsinki-NLP/opus-mt-en-es"
tokenizer_trad = MarianTokenizer.from_pretrained(modelo_trad)
model_trad = MarianMTModel.from_pretrained(modelo_trad)

def traducir_texto(texto, max_length=512):
    oraciones = [texto[i:i+max_length] for i in range(0, len(texto), max_length)]
    resultado = []
    for segmento in oraciones:
        inputs = tokenizer_trad(segmento, return_tensors="pt", truncation=True)
        translated = model_trad.generate(**inputs)
        texto_traducido = tokenizer_trad.decode(translated[0], skip_special_tokens=True)
        resultado.append(texto_traducido)
    return " ".join(resultado)


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# ======================================
# 📄 5. Traducción de documentos en inglés
# ======================================
def traducir_documentos_en_ingles(ruta):
    documentos = []
    for archivo in os.listdir(ruta):
        if archivo.endswith(".pdf"):
            ruta_completa = os.path.join(ruta, archivo)
            loader = PyMuPDFLoader(ruta_completa)
            docs = loader.load()
            for doc in docs:
                texto_original = doc.page_content
                texto_traducido = traducir_texto(texto_original)
                doc.page_content = texto_traducido
                documentos.append(doc)
    return documentos


In [ ]:
# ======================================
# 📦 6. Ejecutamos la carga total del corpus
# ======================================
documentos_es = cargar_documentos_desde_carpeta([carpetas_es])
documentos_en = traducir_documentos_en_ingles(carpeta_en)

documentos = documentos_es + documentos_en

CHUNKING (División en fragmentos)

In [ ]:
# 📌 VALIDACIONES previas antes del chunking

print("🔎 Validando el corpus final...")

# 1. ¿Qué tipo de objeto es el corpus?
print(f"Tipo de corpus_completo: {type(documentos)}")

# 2. ¿Cuántos documentos contiene?
print(f"📚 Total de documentos: {len(documentos)}")

# 3. ¿Qué tipo de objeto es cada documento?
if documentos:
    print(f"Ejemplo de tipo de documento: {type(documentos[0])}")

# 4. Mostrar los primeros 500 caracteres del primer documento
if documentos and hasattr(documentos[0], "page_content"):
    print("\n📝 Vista previa del primer documento:")
    print(documentos[0].page_content[:500])
else:
    print("⚠️ No se encontró texto en el primer documento.")


🔎 Validando el corpus final...
Tipo de corpus_completo: <class 'list'>
📚 Total de documentos: 420
Ejemplo de tipo de documento: <class 'langchain_core.documents.base.Document'>

📝 Vista previa del primer documento:
Manual de Producción 
Upstream
Tomos 1, 2 y 3


CHUNKING (División en fragmentos)

In [ ]:
# ✅ Importamos la herramienta para dividir documentos en fragmentos/chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# ✅ Configuramos el splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cantidad máxima de caracteres por fragmento
    chunk_overlap=50,      # Cuántos caracteres se solapan entre fragmentos
    separators=["\n\n", "\n", ".", " ", ""],  # Orden de preferencia para cortar texto
)


In [ ]:
# ✅ Aplicar el splitter a todos los documentos
chunks = text_splitter.split_documents(documentos)


In [ ]:
from pathlib import Path

# ✅ Enriquecer y simplificar la metadata de cada chunk
for chunk in chunks:
    metadata = chunk.metadata

    # Extraer nombre de archivo limpio
    file_name = Path(metadata.get("file_path", "desconocido")).stem

    # Agregar nombre simple
    metadata["nombre_documento"] = file_name

    # Agregar categoría manual según carpeta origen
    if "corpus_base" in metadata.get("file_path", ""):
        metadata["categoria"] = "base_tecnica"
        metadata["idioma"] = "es"
    elif "complementos_tecnicos" in metadata.get("file_path", ""):
        metadata["categoria"] = "complemento"
        metadata["idioma"] = "en"
    else:
        metadata["categoria"] = "otro"
        metadata["idioma"] = "desconocido"

    # Limpiar metadata innecesaria
    for campo in ["producer", "creator", "format", "encryption", "trapped", "moddate", "creationdate", "title", "author", "subject", "keywords"]:
        metadata.pop(campo, None)


In [ ]:
# Validar resultados del chunking
print(f"📄 Total de chunks generados: {len(chunks)}")
print("\n📝 Primer chunk:")
print(chunks[0].page_content[:500])


📄 Total de chunks generados: 2747

📝 Primer chunk:
Manual de Producción 
Upstream
Tomos 1, 2 y 3


In [ ]:
# Revisar la metadata de los primeros 3 chunks
for i, chunk in enumerate(chunks[:3]):
    print(f"\n🧾 Chunk {i+1} - Metadata:")
    print(chunk.metadata)



🧾 Chunk 1 - Metadata:
{'source': '/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base/Manual de Producción Upstream YPF.pdf', 'file_path': '/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base/Manual de Producción Upstream YPF.pdf', 'total_pages': 393, 'modDate': "D:20130808171109-03'00'", 'creationDate': "D:20130806131834-03'00'", 'page': 0, 'nombre_documento': 'Manual de Producción Upstream YPF', 'categoria': 'base_tecnica', 'idioma': 'es'}

🧾 Chunk 2 - Metadata:
{'source': '/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base/Manual de Producción Upstream YPF.pdf', 'file_path': '/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base/Manual de Producción Upstream YPF.pdf', 'total_pages': 393, 'modDate': "D:20130808171109-03'00'", 'creationDate': "D:20130806131834-03'00'", 'page': 1, 'nombre_documento': 'Manual de Producción Upstream YPF', 'categoria': 'base_tecnica', 'idioma': 'es'}

🧾 Chunk 3 - Metadata:
{

EMBEDDINGS

In [ ]:
# ==========================
# Paso 3.1: Cargar modelo de embeddings en español
# ==========================

from langchain_huggingface import HuggingFaceEmbeddings # Updated import

# Definir el modelo de embeddings
modelo_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

/tmp/ipython-input-3578164995.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  modelo_embeddings = HuggingFaceEmbeddings(


In [ ]:

# ===============================================
# 🧠 Paso 3.2: Vectorización de Chunks con ChromaDB
# ===============================================

# Ruta donde se almacenará la base vectorial
persistencia_vectores = "db_vectores"

# Crear base de datos vectorial con los embeddings
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=modelo_embeddings,
    persist_directory="db_vectores"
)

# Guardar la base persistente en disco
chroma_db.persist()

# Validación visual
print("✅ Base de datos vectorial creada con éxito y guardada en:", persistencia_vectores)



✅ Base de datos vectorial creada con éxito y guardada en: db_vectores


/tmp/ipython-input-1970422414.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist()


In [ ]:
# 📌 Paso 2.1 – Ingesta y traducción automática de documentos PDF

# ========================
# ✅ Librerías necesarias
# ========================
!pip install --upgrade langchain langchain-community langchain-core langchain-text-splitters transformers pypdf pymupdf  # Upgrade core libraries
!pip install -U huggingface_hub langchain-huggingface # Install compatible huggingface_hub and langchain-huggingface
from langchain_community.document_loaders import PyMuPDFLoader  # Para cargar PDFs

from transformers import pipeline                                # Para traducir texto
import os                                                        # Para acceder a archivos en disco


!pip install chromadb
from langchain_community.vectorstores import Chroma

In [ ]:
# ====================================================
# 🧠 Paso 5.1: Cargar base vectorial persistente y preparar el Retriever
# ====================================================

from langchain_community.vectorstores import Chroma

# 🔄 Ruta donde guardaste la base de datos vectorial
persistencia_vectores = "db_vectores"

# 🗃️ Cargar la base vectorial persistente desde el disco
chroma_db = Chroma(
    persist_directory=persistencia_vectores,
    embedding_function=modelo_embeddings
)

# 🔍 Crear un Retriever para realizar búsquedas por similitud
retriever = chroma_db.as_retriever(search_kwargs={"k": 3})

# ✅ Validación
print("✅ Retriever creado correctamente. Listo para recuperar chunks similares.")


✅ Retriever creado correctamente. Listo para recuperar chunks similares.


/tmp/ipython-input-1341945274.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  chroma_db = Chroma(


In [ ]:
# =============================================
# 🟪 Paso 5.2: Consulta de Prueba y Recuperación
# =============================================

from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

# 🧠 Cargamos la base vectorial persistida
chroma_db = Chroma(
    persist_directory="db_vectores",
    embedding_function=modelo_embeddings
)

# 🔍 Creamos el retriever (mecanismo de recuperación)
retriever = chroma_db.as_retriever(
    search_type="similarity",  # También puedes usar "mmr" (Maximal Marginal Relevance)
    search_kwargs={"k": 3}      # Número de documentos más similares que queremos recuperar
)

# 🧪 Definimos una pregunta de prueba
pregunta_prueba = "¿Qué es un reservorio no convencional?"

# 🔄 Recuperamos los documentos más relevantes
resultados = retriever.get_relevant_documents(pregunta_prueba)

# 🖨️ Mostramos los resultados
print("📌 Resultados de la recuperación:\n")
for i, doc in enumerate(resultados, 1):
    print(f"🔹 Documento {i}:")
    print(doc.page_content[:500])  # Muestra los primeros 500 caracteres
    print("📎 Metadata:", doc.metadata)
    print("-" * 80)


ModuleNotFoundError: No module named 'langchain.chains'